# IndoBenchmark: POSP

Named-entity recognition (NER) is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into pre-defined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc. [[Wikipedia: Named Entity Recognition]](https://en.wikipedia.org/wiki/Named-entity_recognition)

We will try to finetune the bert-base-indonesian-522M for the Named Entity Recognition (NER) task. For this purpose we will use the [NERGRIT Corpus](https://github.com/grit-id/nergrit-corpus) which contains 321.757 lines of train, 66.974 lines of test and 64.208 lines of valid dataset. It uses Inside-Outside-Beginning (IOB) format where each line is composed of a word and its label/category. 

The [NERGRIT Corpus](https://github.com/grit-id/nergrit-corpus) is a very valueable dataset for indonesian NLP researcher. Unfortunately there are many typos or errors on the labels, so I spent some times to analyse the errors, make corrections and report the [issue to their Github repository](https://github.com/grit-id/nergrit-corpus/issues/1). Since the license allows us to redistribute the dataset, I will also publish the original dataset including its corrections. Currently the dataset is only available per [request](https://ner.grit.id/index.php/front/about) (klick the "Get NERGRIT Corpus").


## Transformers or Simpletransformers?

We will use simpletransformers in this case to simplify the training and inferencing

In [1]:
from simpletransformers.ner import NERModel, NERArgs
import pandas as pd
import logging
import sys

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

We use the corrected dataset which has less lines than the originals (train: 309203, valid: 61680, test: 64568)

In [34]:
data_dir = "/dataset/indonlu/bapos_pos-idn"
file_train = f'{data_dir}/train_preprocess.txt'
file_valid = f'{data_dir}/valid_preprocess.txt'
file_train_valid = f'{data_dir}/train_preprocess.txt'
file_test = f'{data_dir}/test_preprocess_masked_label.txt'
#file_labels_map = f'{data_dir}/labels-map.csv'

The Simpletransformers requires the dataset either as pandas dataframe with following column/format: **sentence_id**, **words**, **labels**, or Text file in CoNLL format. The **sentence_id** is consecutive number determines which words belong to a given sentence. I.e. the words from the same sequence should be assigned the same unique sentence_id.

In [4]:
# Function to read ner file in connl format and return a DataFrame with columns: sentence_id, words, labels
def get_pos_data(filename, labels_map=None):
    word_list = []
    sentence_counter = 0
    with open(filename) as fp:
        for cnt, line in enumerate(fp):
            try:
                texts = line.split()
                if len(texts) != 0:
                    word, label = ' '.join(texts[0:-1]), texts[-1]
                    if labels_map:
                        label = labels_map[label]
                    word_list.append([sentence_counter, word, label])
                else:
                    sentence_counter += 1
            except:
                print("Unexpected error:", sys.exc_info()[0], cnt, line)
                word_list.append([sentence_counter, "", ""])              
                sentence_counter += 1
                pass
    print(f'read {cnt} lines')
    ner_data = pd.DataFrame(word_list, columns=["sentence_id", "words", "labels"])
    return ner_data

In [5]:
train_data = get_pos_data(file_train)

read 222450 lines


In [6]:
valid_data = get_pos_data(file_valid)

read 26875 lines


In [7]:
test_data = get_pos_data(file_test)

read 27726 lines


In [8]:
len(train_data),len(valid_data),len(test_data)

(214451, 25876, 26698)

In [9]:
labels = list(set(train_data['labels']))

In [10]:
labels.append('I-IN')

In [11]:
labels.sort()

In [12]:
labels

['B-CC',
 'B-CD',
 'B-DT',
 'B-FW',
 'B-IN',
 'B-JJ',
 'B-MD',
 'B-NEG',
 'B-NN',
 'B-NND',
 'B-NNP',
 'B-OD',
 'B-PR',
 'B-PRP',
 'B-RB',
 'B-RP',
 'B-SC',
 'B-SYM',
 'B-UH',
 'B-VB',
 'B-WH',
 'B-X',
 'B-Z',
 'I-CC',
 'I-CD',
 'I-IN',
 'I-JJ',
 'I-NN',
 'I-NND',
 'I-NNP',
 'I-OD',
 'I-PR',
 'I-PRP',
 'I-RB',
 'I-SC',
 'I-SYM',
 'I-UH',
 'I-VB',
 'I-WH',
 'I-X',
 'I-Z']

In [13]:
train_data.iloc[:50]

,sentence_id,words,labels
0,0,Kera,B-NN
1,0,untuk,B-SC
2,0,amankan,B-VB
3,0,pesta,B-NN
4,0,olahraga,I-NN
5,1,Pemerintah,B-NNP
6,1,kota,B-NNP
7,1,Delhi,B-NNP
8,1,mengerahkan,B-VB
9,1,monyet,B-NN


In [14]:
valid_data.head(10)

,sentence_id,words,labels
0,0,Seorang,B-NND
1,0,penduduk,B-NN
2,0,yang,B-SC
3,0,tinggal,B-VB
4,0,dekat,B-JJ
5,0,tempat,B-NN
6,0,kejadian,B-NN
7,0,mengatakan,B-VB
8,0,",",B-Z
9,0,dia,B-PRP


In [15]:
test_data.head(10)

,sentence_id,words,labels
0,0,Pada,B-NNP
1,0,2005,B-NNP
2,0,",",B-NNP
3,0,tembakan,B-NNP
4,0,musuh,B-NNP
5,0,dari,B-NNP
6,0,darat,B-NNP
7,0,ke,B-NNP
8,0,udara,B-NNP
9,0,menjatuhkan,B-NNP


## The Labels

The NERGRIT corpus contains 19 entities, each with Inside- and Beginning-Tag, plus an Outside-Tag. Alltogether become 39 categories. The entities have following meaning:
1. 'CRD' --> Cardinal
1. 'DAT' --> Date
1. 'EVT' --> Event
1. 'FAC' --> Facility
1. 'GPE' --> Geopolitical Entity
1. 'LAW' --> Law Entity (such as Undang-Undang)
1. 'LOC' --> Location
1. 'MON' --> Money
1. 'NOR' --> Political Organization
1. 'ORD' --> Ordinal
1. 'ORG' --> Organization
1. 'PER' --> Person
1. 'PRC' --> Percent
1. 'PRD' --> Product
1. 'QTY' --> Quantity
1. 'REG' --> Religion
1. 'TIM' --> Time
1. 'WOA' --> Work of Art
1. 'LAN' --> Language

## The Training with bert-base-indonesian-522M

Since I have already pre-trained the bert-base with indonesian Wikipedia, I want to try its performance for this task.

In [16]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 64
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/bapos/bert-base-indonesian-1.5G'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [17]:
model_args

NERArgs(adam_epsilon=1e-08, best_model_dir='/output/indonlu/bapos/bert-base-indonesian-522M/best_model', cache_dir='cache_dir/', config={}, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=78, do_lower_case=True, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=True, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=128, model_name=None, model_type=None, multiprocessing_chunksize=500, n_gpu=1, no_cache=False, no_save=False, num_train_epochs=5, output_dir='/output/indonlu/bapos/bert-base-indonesian-522M', overwrite_output_dir=True

In [18]:
model_bert_base = NERModel(
    #"bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
    "bert", "cahya/bert-base-indonesian-1.5G", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-1.5G were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [19]:
# Train the model
model_bert_base.train_model(train_data, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/bapos/bert-base-indonesian-522M.


In [20]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.12274945671856403, 'precision': 0.9656480926702616, 'recall': 0.9671160539264712, 'f1_score': 0.9663815158298689}


In [21]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 8.851375520691391, 'precision': 0.14860248447204968, 'recall': 0.1433815267061203, 'f1_score': 0.14594532769072402}


In [22]:
test_data.iloc[0:30]

,sentence_id,words,labels
0,0,Pada,B-NNP
1,0,2005,B-NNP
2,0,",",B-NNP
3,0,tembakan,B-NNP
4,0,musuh,B-NNP
5,0,dari,B-NNP
6,0,darat,B-NNP
7,0,ke,B-NNP
8,0,udara,B-NNP
9,0,menjatuhkan,B-NNP


In [23]:
preds_list[:5]

[['B-IN',
  'B-CD',
  'B-Z',
  'B-NN',
  'B-NN',
  'B-IN',
  'B-NN',
  'B-IN',
  'B-NN',
  'B-VB',
  'B-NND',
  'B-NN',
  'B-VB',
  'B-NN',
  'B-NNP',
  'B-IN',
  'B-NN',
  'B-SC',
  'B-JJ',
  'B-Z',
  'B-VB',
  'B-CD',
  'B-NN',
  'B-SC',
  'B-VB',
  'B-IN',
  'B-NN',
  'B-PRP',
  'I-PRP',
  'B-Z'],
 ['B-NN',
  'B-NNP',
  'B-MD',
  'B-VB',
  'B-NN',
  'B-IN',
  'B-NNP',
  'B-NNP',
  'B-NNP',
  'B-NNP',
  'B-IN',
  'B-NN',
  'B-VB',
  'B-NN',
  'B-Z',
  'B-PR',
  'B-VB',
  'B-NN',
  'B-NNP',
  'B-CD',
  'B-NNP',
  'B-Z',
  'B-NNP',
  'B-Z'],
 ['B-NNP',
  'B-MD',
  'B-VB',
  'B-IN',
  'B-CD',
  'B-NN',
  'B-NN',
  'B-SC',
  'B-MD',
  'B-VB',
  'B-SC',
  'B-PRP',
  'B-VB',
  'B-IN',
  'B-NNP',
  'B-Z',
  'B-VB',
  'B-NN',
  'B-PR',
  'B-Z'],
 ['B-NN',
  'B-PR',
  'B-MD',
  'B-VB',
  'B-JJ',
  'B-PRP',
  'I-PRP',
  'B-IN',
  'B-WH',
  'B-PRP',
  'B-VB',
  'B-NN',
  'B-PRP',
  'I-PRP',
  'B-SC',
  'B-NN',
  'B-IN',
  'B-NN',
  'B-VB',
  'B-NNP',
  'B-NNP',
  'B-NNP',
  'B-SC',
  'B-NNP',
 

In [24]:
len(preds_list)

1029

In [25]:
def print_result(preds_list, test_data, max_len=10):
    for i in range(len(preds_list)):
        if i>max_len:
            break
        sentence = list(test_data[test_data['sentence_id']==i]['words'])
        for j, word in enumerate(sentence):
            print(f'{i}:{word}\t{preds_list[i][j]}')

def save_result(preds_list, test_data, filename):
    with open(filename, 'w') as out_file:
        out_file.write(f'index,label\n')
        index = 0
        for i in range(len(preds_list)):
            label = str(preds_list[i])
            out_file.write(f'{index},"{label}"\n')
            index += 1


In [26]:
#output_dir = "/output/indonlu/nerp"
output_fn = f'{model_args.output_dir}/result.txt'

In [27]:
output_fn

'/output/indonlu/bapos/bert-base-indonesian-522M/result.txt'

In [28]:
print_result(preds_list, test_data, 6)

0:Pada	B-IN
0:2005	B-CD
0:,	B-Z
0:tembakan	B-NN
0:musuh	B-NN
0:dari	B-IN
0:darat	B-NN
0:ke	B-IN
0:udara	B-NN
0:menjatuhkan	B-VB
0:sebuah	B-NND
0:pesawat	B-NN
0:angkut	B-VB
0:militer	B-NN
0:Inggris	B-NNP
0:di	B-IN
0:daerah	B-NN
0:yang	B-SC
0:sama	B-JJ
0:,	B-Z
0:menewaskan	B-VB
0:seluruh	B-CD
0:orang	B-NN
0:yang	B-SC
0:berada	B-VB
0:di	B-IN
0:dalam	B-NN
0:-	B-PRP
0:nya	I-PRP
0:.	B-Z
1:Polisi	B-NN
1:Israel	B-NNP
1:akan	B-MD
1:memulai	B-VB
1:penyelidikan	B-NN
1:terhadap	B-IN
1:Perdana	B-NNP
1:Menteri	B-NNP
1:Ehud	B-NNP
1:Olmert	B-NNP
1:atas	B-IN
1:dugaan	B-NN
1:melakukan	B-VB
1:korupsi	B-NN
1:,	B-Z
1:demikian	B-PR
1:dilaporkan	B-VB
1:televisi	B-NN
1:Aruz	B-NNP
1:10	B-CD
1:Israel	B-NNP
1:,	B-Z
1:Selasa	B-NNP
1:.	B-Z
2:Olmert	B-NNP
2:akan	B-MD
2:diinterogasi	B-VB
2:mengenai	B-IN
2:sejumlah	B-CD
2:kasus	B-NN
2:korupsi	B-NN
2:yang	B-SC
2:mungkin	B-MD
2:dilakukan	B-VB
2:ketika	B-SC
2:ia	B-PRP
2:pulang	B-VB
2:dari	B-IN
2:China	B-NNP
2:,	B-Z
2:kata	B-VB
2:televisi	B-NN
2:tersebut	B-PR
2:.	B-Z
3:P

In [29]:
save_result(preds_list, test_data, output_fn)

In [30]:
!head $output_fn

index,label
0,"['B-IN', 'B-CD', 'B-Z', 'B-NN', 'B-NN', 'B-IN', 'B-NN', 'B-IN', 'B-NN', 'B-VB', 'B-NND', 'B-NN', 'B-VB', 'B-NN', 'B-NNP', 'B-IN', 'B-NN', 'B-SC', 'B-JJ', 'B-Z', 'B-VB', 'B-CD', 'B-NN', 'B-SC', 'B-VB', 'B-IN', 'B-NN', 'B-PRP', 'I-PRP', 'B-Z']"
1,"['B-NN', 'B-NNP', 'B-MD', 'B-VB', 'B-NN', 'B-IN', 'B-NNP', 'B-NNP', 'B-NNP', 'B-NNP', 'B-IN', 'B-NN', 'B-VB', 'B-NN', 'B-Z', 'B-PR', 'B-VB', 'B-NN', 'B-NNP', 'B-CD', 'B-NNP', 'B-Z', 'B-NNP', 'B-Z']"
2,"['B-NNP', 'B-MD', 'B-VB', 'B-IN', 'B-CD', 'B-NN', 'B-NN', 'B-SC', 'B-MD', 'B-VB', 'B-SC', 'B-PRP', 'B-VB', 'B-IN', 'B-NNP', 'B-Z', 'B-VB', 'B-NN', 'B-PR', 'B-Z']"
3,"['B-NN', 'B-PR', 'B-MD', 'B-VB', 'B-JJ', 'B-PRP', 'I-PRP', 'B-IN', 'B-WH', 'B-PRP', 'B-VB', 'B-NN', 'B-PRP', 'I-PRP', 'B-SC', 'B-NN', 'B-IN', 'B-NN', 'B-VB', 'B-NNP', 'B-NNP', 'B-NNP', 'B-SC', 'B-NNP', 'B-VB', 'B-NN', 'B-NN', 'B-NNP', 'B-Z']"
4,"['B-NNP', 'B-CD', 'B-NND', 'B-VB', 'B-VB', 'B-IN', 'B-NN', 'B-NN', 'B-CC', 'B-NEG', 'B-MD', 'B-VB', 'B-NN', 'B-Z']"
5,"['B-PR

In [31]:
result = pd.read_csv(output_fn, names=['index','label']).set_index('index')

In [33]:
result.iloc[0:20]

,label
index,
index,label
0,"['B-IN', 'B-CD', 'B-Z', 'B-NN', 'B-NN', 'B-IN'..."
1,"['B-NN', 'B-NNP', 'B-MD', 'B-VB', 'B-NN', 'B-I..."
2,"['B-NNP', 'B-MD', 'B-VB', 'B-IN', 'B-CD', 'B-N..."
3,"['B-NN', 'B-PR', 'B-MD', 'B-VB', 'B-JJ', 'B-PR..."
4,"['B-NNP', 'B-CD', 'B-NND', 'B-VB', 'B-VB', 'B-..."
5,"['B-PRP', 'B-RB', 'B-VB', 'B-NN', 'B-PR', 'B-Z']"
6,"['B-NN', 'B-NN', 'B-NN', 'I-NN', 'B-NNP', 'B-N..."
7,"['B-NNP', 'B-Z', 'B-SC', 'B-OD', 'B-NND', 'B-V..."


### BaPOS 2 (train+valid)

In [35]:
train_data_all = pd.concat([train_data, valid_data], ignore_index=True)

In [37]:
len(train_data_all), len(train_data), len(valid_data), 

(240327, 214451, 25876)

In [39]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/bapos/bert-base-indonesian-1.5G-all'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [40]:
model_bert_base = NERModel(
    "bert", "cahya/bert-base-indonesian-1.5G", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-1.5G were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [41]:
# Train the model
model_bert_base.train_model(train_data_all, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/bapos/bert-base-indonesian-1.5G-all.


In [42]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.01752241707826033, 'precision': 0.9928839849684177, 'recall': 0.9935592271072529, 'f1_score': 0.9932214912719202}


In [43]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 9.501249583192575, 'precision': 0.14609796837975372, 'recall': 0.1408719754288711, 'f1_score': 0.1434373867775214}


In [44]:
save_result(preds_list, test_data, output_fn)

In [45]:
output_fn

'/output/indonlu/bapos/bert-base-indonesian-522M/result.txt'

In [28]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/bert-base-indonesian'
model_args.best_model_dir = '/output/indonlu/bert-base-indonesian/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [31]:
model_bert_base = NERModel(
    "bert", "/output/indonlu/bert-base-indonesian/best_model",  args=model_args
)

In [32]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.12883393300904167, 'precision': 0.7910662824207493, 'recall': 0.8293051359516617, 'f1_score': 0.8097345132743363}


In [33]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.7154410238619204, 'precision': 0.0, 'recall': 0, 'f1_score': 0}


In [34]:
save_result(preds_list, test_data_2, output_fn)

### Nergrit 1 vs Nergrit 2

In [41]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/indonlu/bert-base-indonesian'
model_args.best_model_dir = '/output/indonlu/bert-base-indonesian/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels
model_args.do_lower_case = True

In [42]:
model_bert_base = NERModel(
    "bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [43]:
# Train the model
model_bert_base.train_model(train_data_1, eval_data=valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/bert-base-indonesian.


In [44]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.3372342694136832, 'precision': 0.6912378303198887, 'recall': 0.7507552870090635, 'f1_score': 0.7197682838522811}


In [39]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.3447435959069817, 'precision': 0.6963276836158192, 'recall': 0.7447129909365559, 'f1_score': 0.7197080291970803}


In [ ]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_bert_base.eval_model(test_data)

In [ ]:
train_data_3 = pd.concat([train_data_1, train_data_2], ignore_index=True)

### (Nergrit 1 +  Nergrit 2) vs Nergrit 2

In [45]:
train_data_3 = pd.concat([train_data_1, train_data_2], ignore_index=True)

In [46]:
len(train_data_3),len(train_data_1),len(train_data_2)

(365416, 309206, 56210)

In [47]:
train_data_1.head()

,sentence_id,words,labels
0,0,Indonesia,B-PLACE
1,0,mengekspor,O
2,0,produk,O
3,0,industri,O
4,0,skala,O


In [48]:
last_si = train_data_1.iloc[-1]['sentence_id']

In [55]:
train_data_tmp = train_data_2

In [63]:
train_data_tmp['sentence_id'] = 100

In [80]:
train_data_tmp.head()

,sentence_id,words,labels
0,12552,Kontribusinya,O
1,12552,terhadap,O
2,12552,industri,O
3,12552,musik,O
4,12552,telah,O


In [67]:
train_data_2.head()

,sentence_id,words,labels
0,0,Kontribusinya,O
1,0,terhadap,O
2,0,industri,O
3,0,musik,O
4,0,telah,O


In [79]:
for i, row in train_data_2.iterrows():
    train_data_tmp.loc[i, 'sentence_id'] = train_data_2.iloc[i]['sentence_id'] + last_si + 1
    #print(i, train_data_tmp.iloc[i]['sentence_id'] )

In [81]:
train_data_3 = pd.concat([train_data_1, train_data_tmp], ignore_index=True)

In [82]:
train_data_3.iloc[309200: 309220]

,sentence_id,words,labels
309200,12551,di,O
309201,12551,38,O
309202,12551,negara,O
309203,12551,bagian,O
309204,12551,lainnya,O
309205,12551,.,O
309206,12552,Kontribusinya,O
309207,12552,terhadap,O
309208,12552,industri,O
309209,12552,musik,O


In [83]:
model_bert_base = NERModel(
    "bert", "cahya/bert-base-indonesian-522M", labels=labels, args=model_args
)

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesia

In [84]:
# Train the model
model_bert_base.train_model(train_data_3, eval_data=valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/sysadmin/wirawan/miniconda3/envs/transformers-cuda9/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to /output/indonlu/bert-base-indonesian.


In [85]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_bert_base.eval_model(valid_data_2)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.1967056412939672, 'precision': 0.7793904208998549, 'recall': 0.8111782477341389, 'f1_score': 0.7949666913397484}


In [24]:
preds_list

[['B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'B-ORGANISATION', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PERSON',
  'I-PERSON',
  'I-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORGANISATION',
  'I-ORGANISATION',
  'O',
  'O',
  'O',
  'B-PLACE',
  'O',
  'B-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  

In [33]:
list(test_data[test_data['sentence_id']==0]['words'])

['Joetata',
 'Hadihardaja',
 'dan',
 'dihadiri',
 'oleh',
 'Rektor',
 'Undip',
 'Prof',
 '.']

In [32]:
for i in range(len(preds_list)):
    sentence = list(test_data[test_data['sentence_id']==i]['words'])
    for j, word in enumerate(sentence):
        print(word, preds_list[i][j])
    if i>10:
        break

NameError: name 'test_data' is not defined

In [28]:
for i, row in test_data.iterrows():
    print(i, row['words'], preds_list[row['sentence_id']])
    
    
    for j in row['words']:
    if i>10:
        break

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 1
10 1
11 1


The result (F1-score: 80.17 %) is quite similar with the F1-score NERGRIT has achieved (about 80%). 
Last week I got very low F1-score (about 60%), I was disappointed because it was much lower then the F1-score achieved by NERGRIT team. It turned out that the model was trained incorrectly, I trained the bert-base-indonesian-522M as if it is cased model (this is the default configuration). After I enabled the lowercase in the configuratin (model_args.do_lower_case = True), the F1-score is much better.


## The Training with xlm-roberta-base

I tried a multilanguage model from Facebook: XLM-Roberta-base which was pre-trained on 2.5TB of dataset.

In [19]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 128
model_args.evaluate_during_training = True
model_args.output_dir = '/output/ner/xlm-roberta-base'
model_args.best_model_dir = '/output/ner/xlm-roberta-base/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels

In [20]:
model_xlmroberta_base = NERModel(
    "xlmroberta", "xlm-roberta-base", labels=labels, args=model_args
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [21]:
# Train the model
model_xlmroberta_base.train_model(train_data, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of xlmroberta model complete. Saved to /output/ner/xlm-roberta-base.


In [22]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_xlmroberta_base.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.1866366565582298, 'precision': 0.8336475023562677, 'recall': 0.8462090408993064, 'f1_score': 0.8398813056379822}


In [23]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_xlmroberta_base.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.18713697187486106, 'precision': 0.8234437975817286, 'recall': 0.8321077044914583, 'f1_score': 0.8277530808620787}


### Result

The result is great, F1-score 82.8%

## The Training with xlm-roberta-large

Then I tried a second multilanguage model from Facebook: XLM-Roberta-large

In [24]:
# Configure the model
model_args = NERArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.output_dir = '/output/ner/xlm-roberta-large'
model_args.best_model_dir = '/output/ner/xlm-roberta-large/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.labels_list=labels

In [25]:
model_xlmroberta_large = NERModel(
    "xlmroberta", "xlm-roberta-large", labels=labels, args=model_args
)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

In [26]:
# Train the model
model_xlmroberta_large.train_model(train_data, eval_data=valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model: Training of xlmroberta model complete. Saved to /output/ner/xlm-roberta-large.


In [27]:
# Evaluate the model with valid dataset
result, model_outputs, preds_list = model_xlmroberta_large.eval_model(valid_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.21224965298888349, 'precision': 0.8447300165055411, 'recall': 0.8568524276488878, 'f1_score': 0.8507480408454049}


In [28]:
# Evaluate the model with test dataset
result, model_outputs, preds_list = model_xlmroberta_large.eval_model(test_data)

INFO:simpletransformers.ner.ner_model: Converting to features started.


INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.21842414552079087, 'precision': 0.836591086786552, 'recall': 0.8473809254440547, 'f1_score': 0.8419514388489209}


### Result

Again, the result is great, it achieved F1-score of 84.19%. It is 4 percent better than the bert-base-indonesian-522M. Maybe  my LM needs more data for pre-training

## Predict some Samples

In [29]:
# Make predictions with the model
texts = [
    "Gubernur Bank Indonesia Agus Martowardojo bersama jajaran deputi Gubernur Bank Indonesia menggelar konferensi pers usai Rapat Dewan Gubernur di Bank Indonesia, Jakarta, Kamis (17/5/2015)",
    "Selama 24 jam puncak Mahameru di Malang kebanjiran pendaki dari Wina",
]

In [30]:
predictions, raw_outputs = model_bert_base.predict(texts)
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


[[{'Gubernur': 'B-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'Agus': 'B-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'B-ORG'},
  {'Gubernur': 'I-ORG'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVT'},
  {'pers': 'I-EVT'},
  {'usai': 'O'},
  {'Rapat': 'B-EVT'},
  {'Dewan': 'I-EVT'},
  {'Gubernur': 'I-EVT'},
  {'di': 'O'},
  {'Bank': 'B-ORG'},
  {'Indonesia,': 'I-ORG'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}],
 [{'Selama': 'O'},
  {'24': 'B-QTY'},
  {'jam': 'I-QTY'},
  {'puncak': 'B-LOC'},
  {'Mahameru': 'I-LOC'},
  {'di': 'O'},
  {'Malang': 'B-GPE'},
  {'kebanjiran': 'O'},
  {'pendaki': 'O'},
  {'dari': 'O'},
  {'Wina': 'B-GPE'}]]

In [31]:
predictions, raw_outputs = model_xlmroberta_base.predict(texts)
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


[[{'Gubernur': 'B-NOR'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'Agus': 'B-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'O'},
  {'Gubernur': 'I-NOR'},
  {'Bank': 'I-NOR'},
  {'Indonesia': 'I-NOR'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVT'},
  {'pers': 'I-EVT'},
  {'usai': 'O'},
  {'Rapat': 'B-EVT'},
  {'Dewan': 'I-EVT'},
  {'Gubernur': 'I-EVT'},
  {'di': 'O'},
  {'Bank': 'B-ORG'},
  {'Indonesia,': 'I-ORG'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}],
 [{'Selama': 'O'},
  {'24': 'B-QTY'},
  {'jam': 'I-QTY'},
  {'puncak': 'B-LOC'},
  {'Mahameru': 'I-LOC'},
  {'di': 'O'},
  {'Malang': 'B-GPE'},
  {'kebanjiran': 'O'},
  {'pendaki': 'O'},
  {'dari': 'O'},
  {'Wina': 'B-GPE'}]]

In [32]:
predictions, raw_outputs = model_xlmroberta_large.predict(texts)
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


[[{'Gubernur': 'B-ORG'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'Agus': 'B-PER'},
  {'Martowardojo': 'I-PER'},
  {'bersama': 'O'},
  {'jajaran': 'O'},
  {'deputi': 'O'},
  {'Gubernur': 'B-ORG'},
  {'Bank': 'I-ORG'},
  {'Indonesia': 'I-ORG'},
  {'menggelar': 'O'},
  {'konferensi': 'B-EVT'},
  {'pers': 'I-EVT'},
  {'usai': 'O'},
  {'Rapat': 'B-EVT'},
  {'Dewan': 'I-EVT'},
  {'Gubernur': 'I-EVT'},
  {'di': 'O'},
  {'Bank': 'B-ORG'},
  {'Indonesia,': 'I-ORG'},
  {'Jakarta,': 'B-GPE'},
  {'Kamis': 'B-DAT'},
  {'(17/5/2015)': 'I-DAT'}],
 [{'Selama': 'O'},
  {'24': 'B-QTY'},
  {'jam': 'I-QTY'},
  {'puncak': 'B-LOC'},
  {'Mahameru': 'I-LOC'},
  {'di': 'O'},
  {'Malang': 'B-GPE'},
  {'kebanjiran': 'O'},
  {'pendaki': 'O'},
  {'dari': 'O'},
  {'Wina': 'B-GPE'}]]